# Intermediate SQL

## CASE statements
Contains a WHEN, THEN, and ELSE statement, finished with END
```SQL
 CASE WHEN x = 1 THEN "a" 
     WHEN x = 2 THEN "b"
     ELSE "c" END AS new_column
```

### Basic CASE statements
The European Soccer Database contains data about matches from 11 European countries from the 2011-2015 match seasons. In total, you will be working with data from 12,800 matches!

You'll notice that in the match table, there are two columns called `hometeam_id` and `awayteam_id`. Both ID's can join to the teamapiid column in the team table, but you cannot perform a join on both columns at the same time.

You can use a CASE statement to identify which teams played in a match. However, your first step is to determine which IDs correspond to the FC Schalke 04 and FC Bayern Munich teams by querying the team table. You will then use these IDs to examine matches played by these teams in the match table.

**match**

`
id	country_id	season	stage	date	hometeam_id	awayteam_id	home_goal	away_goal
757 	 1	     2011/2012	 1	2011-07-29	1773	     8635	       2	          1
758	  1	     2011/2012	 1	2011-07-30	9998	     9985	       1	          1
759	  1    	 2011/2012	 1	2011-07-30	9987	     9993	       3	          1
.............................`


**team**

`
id	team_api_id	team_long_name	team_short_name
1	  9987	        KRC Genk	         GEN
2	  9993	        Beerschot AC	     BAC
3	  10000	       SV Zulte-Waregem	 ZUL
.............................
`

   **Instructions**
   
**1.1**- Create a query that gives you the team IDs for FC Schalke 04 and FC Bayern Munich.

```SQL
SELECT
	-- Select the team long name and team API id
	team_long_name, team_api_id
	
FROM team 
-- Only include FC Schalke 04 and FC Bayern Munich
WHERE team_long_name IN ('FC Schalke 04', 'FC Bayern Munich');

```
`#OUTPUT
team_long_name	      team_api_id
FC Bayern Munich	     9823
FC Schalke 04	        10189
.............................`





**2.1** Create a CASE statement that identifies whether a match played in Germany included Bayern Munich, Schalke 04, or neither.<br>
**2.2** .Count the id column to determine the number of matches played by each team.<br>
**2.3** Group the query by the CASE statement.


```SQL
-- Identify if a match included Bayern Munich, Schalke 04, or neither
SELECT 
	CASE WHEN hometeam_id = 10189 OR awayteam_id = 10189 THEN 'FC Schalke 04'
        WHEN  hometeam_id = 9823  OR awayteam_id = 9823 THEN 'FC Bayern Munich'
         ELSE 'Other' END AS team_name,
	COUNT(id) AS matches
FROM match
WHERE country_id = 7809
-- Group by the CASE statement
GROUP BY  
	CASE WHEN hometeam_id = 10189 OR awayteam_id = 10189 THEN 'FC Schalke 04'
        WHEN  hometeam_id = 9823  OR awayteam_id = 9823 THEN 'FC Bayern Munich'
         ELSE 'Other' END
```
`OUTPUT
team_name	          matches
Other	                960
FC Schalke 04	        136
FC Bayern Munich     	128
.............................`


### CASE statements comparing column values

 **QUERY:** Querying a list of all matches in the 2011/2012 season where Barcelona was the home team. You will do this using a CASE statement that compares the values of two columns to create a new group -- wins, losses, and ties. The match table contains two columns -- home team ID, and away team ID, to join with the team table. Since you can only join team to one column at a time, you will create two queries in this and the next exercise to examine Barcelona's performance.

```SQL
SELECT m.date, t.team_long_name AS opponent
    CASE WHEN m.home_goal >  m.away_goal THEN 'Barcelone Win'
         WHEN m.home_goal < m.away_goal THEN 'Barcelona Loss :('
         ELSE 'Tie' END AS outcome

FROM match AS m
LEFT JOIN team AS t
ON m.awayteam_id = t.team_api_id
-- Filter for 2012/2013 and Barcelona as the home team
WHERE m.season = '2011/2012' AND m.hometeam_id = 8634
-- Order by the date, with most recent first
ORDER BY m.date ASC; 

```
`OUTPUT
2012-04-10	Getafe CF	 Barcelona win!
2012-04-21	Real Madrid   CFBarcelona loss :(
2012-05-02	Málaga CF	 Barcelona win!
2012-05-05	RCD           Espanyol
.............................`




```SQL
-- Select matches where Barcelona was the away team
SELECT  
	m.date,
	t.team_long_name AS opponent,
	CASE WHEN m.home_goal < m.away_goal THEN 'Barcelona win!'
         WHEN m.home_goal > m.away_goal THEN 'Barcelona loss :(' 
         ELSE 'Tie' END AS outcome
FROM match AS m
-- Join the team table to match
LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id
WHERE  m.season = '2011/2012' AND m.awayteam_id = 8634
ORDER BY m.date;
```
`OUTPUT
2011-09-10	Real Sociedad	             Tie
2011-09-21	Valencia CF	               Tie
2011-10-02	Real Sporting de Gijón	    Barcelona win!
.............................`

### CASE WHEN things to look out for
 **QUERY:**  Barcelona and Real Madrid have been rival teams for more than 80 years. Matches between these two teams are given the name El Clásico (The Classic). In this exercise, you will query a list of matches played between these two rivals.


```SQL
SELECT 
	date,
	CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as home,
	CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as away,
	-- Identify all 5 possible match outcomes
	CASE WHEN home_goal > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'
          WHEN home_goal > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'
          WHEN home_goal < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'
          WHEN home_goal < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'
          ELSE 'Tie!' END AS outcome
FROM match
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);
```
`OUTPUT
date	      home	           away	          outcome
2011-12-10	Real Madrid CF	FC Barcelona	  Barcelona win!
2012-04-21	FC Barcelona	  Real Madrid CF	Real Madrid win!
2013-03-02	Real Madrid CF	FC Barcelona	  Real Madrid win!
2012-10-07	FC Barcelona	  Real Madrid CF	Tie!
..........................................................`

We will use the match and `country` table which showes below.

`country`<br>
`id	    name
1	     Belgium
1729	  England
4769	  France
7809	  Germany
10257	 Italy
13274	 Netherlands
15722	 Poland
.............................`
<br>
<br>
`match` <br>
`id	country_id	season	  stage	    date	hometeam_id	awayteam_id	home_goal	away_goal
757	1	       2011/2012	   1	  2011-07-29	1773	     8635	          2	       1
758	1	       2011/2012	   1	  2011-07-30	9998	     9985	          1	       1
759	1	       2011/2012	   1	  2011-07-30	9987	     9993	          3	       1
760	1	       2011/2012	   1	  2011-07-30	9991	     9984	          0	       1
761	1	       2011/2012	   1	  2011-07-30	9994	     10000	         0	       0
762	1	       2011/2012	   1	  2011-07-30	8571	     9989	          1	       1
.......................................................................................`

 **QUERY:** You will examine the number of matches played in 3 seasons within each country listed in the database. This is much easier to explore with each season's matches in separate columns. Using the country and match table, count the number of matches played in each country during the 2013, 2014, and 2015 seasons
```SQL
SELECT 
	c.name AS country,
    -- Count all games from the 2013, 2014, and 2015 seasons
	COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
	COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,
	COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015
-- Join the match table onto the country table
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY c.name;

```

 **QUERY:**  You will use the country and match table to sum the number of matches won by the home team in each country during the 2013, 2014, and 2015 soccer seasons.
```SQL
SELECT 
	c.name AS country,
    -- Sum the total records in each season where the home team won
	SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2012_2013,
 	SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > away_goal
        THEN 1 ELSE 0 END) AS matches_2013_2014,
	SUM(CASE WHEN m.season = '2014/2015' AND home_goal > away_goal
	    THEN 1 ELSE 0 END) AS matches_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY c.name;
```

  **QUERY:** You will examine data about the number and percentage of wins, loses, and ties in each country.
```SQL
SELECT 
	c.name AS country,
    -- Round the percentage of tied games to 2 decimal points
	ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1
			 WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0
			 END)*100,2) AS ties_2014,
	ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1
			 WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0
			 END)*100,2) AS ties_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY c.name;

```

## Short and Simple Subqueries

### Filtering using scalar subqueries

 **QUERY:** You will generate a list of teams whose total goals scored (for both teams in total) is more than 3 times the average for games in the 2013/2014 season.
 
 ```SQL
 SELECT 
	-- Select the date, home team and away team ID/goals scored
    date,
	hometeam_id,
	awayteam_id,
	home_goal,
	away_goal
FROM  match
WHERE 
	season = '2013/2014'
-- Filter for matches where total goals exceeds 3x the average
	AND (home_goal + away_goal) >
        (SELECT 3 * AVG(home_goal + away_goal)
         FROM match); 
 
 ```
 `OUTPUT 
 date	hometeam_id	awayteam_id	home_goal	away_goal
2013-12-14	8456	     9825	        6	      3
2014-03-22	8344	     8650	        3	      6
2013-10-30	8633	     8302	        7	      3
.............................`


### Filtering using a subquery with a list
**QUERY** You will generate a list of teams that do not have any home matches listed in the database.
```SQL
SELECT 
	-- Select the team long and short names
	team_long_name,
	team_short_name
FROM team
WHERE 
	-- Filter for teams that are NOT IN the subquery
	team_api_id NOT IN
     (SELECT DISTINCT hometeam_id  FROM match);
```
`OUTPUT
team_long_name	team_short_name
FCV Dender EH	 DEN
KSV Roeselare	 ROS
Tubize	        TUB
.............................`

### Filtering with more complex subquery conditions
**QUERY**  Based on the single subquery created, you aren't able to determine if the 52 teams listed played any matches at all. In order to do this, you would need to construct a second subquery in the WHERE clause also eliminating teams whose ID's were in a list of away games.
```SQL
SELECT
	-- Select the team long and short names
	team_long_name,
	team_short_name
FROM team
WHERE 
	-- Filter by the 2 subquery lists of home and away team IDs
	team_api_id NOT IN
   		(SELECT DISTINCT hometeam_id FROM match)
    AND team_api_id NOT IN
  	    (SELECT DISTINCT awayteam_id FROM match);
```

### Subqueries in FROM

 **QUERY** you will be creating a table of values in a subquery to include in a main query, and join it to a table from the database.
```SQL
SELECT
	-- Select country name, season, and the count of match IDs
    c.name AS country,
    COUNT(m.id) AS matches
FROM country AS c
-- Inner join the subquery onto country
-- Select the country id and match id columns
INNER  JOIN (SELECT country_id , id 
           FROM match
           -- Filter the subquery by matches with 10 or more goals
           WHERE (home_goal + away_goal) >= 10) AS m
ON c.id = m.country_id
GROUP BY c.name;
```
`OUTPUT
country	    matches
Netherlands	    1
Spain	          4
Germany            1
England            3`


**QUERY** In the previous exercise, you found that England, Netherlands, Germany and Spain were the only countries that had matches in the database where 10 or more goals were scored overall. Let's find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were home vs. away goals.
```SQL
SELECT
	-- Select country, date, home, and away goals from the subquery
    country,
    date,
    home_goal,
    away_goal
FROM
	-- Select country name, date, and total goals in the subquery
	(SELECT c.name AS country, 
     	    m.date, 
     		m.home_goal, 
     		m.away_goal,
           (m.home_goal + m.away_goal) AS total_goals
    FROM match AS m
    LEFT JOIN country AS c
    ON m.country_id = c.id) AS subq
-- Filter by total goals scored in the main query
WHERE total_goals >= 10;
```
`OUTPUT
country	    date	   home_goal	away_goal
England 	   2011-08-28	8	     2
England	    2012-12-29	7	     3
England	    2013-05-19	5	     5
Germany	    2013-03-30	9	     2
Netherlands	2011-11-06	6	     4
Spain	      2013-10-30	7	     3
Spain	      2015-04-05	9	     1
Spain	      2015-05-23	7	     3`

The following table shows the example of league table.

`league 
id	  country_id	name
1	     1	    Belgium Jupiler League
1729	 1729	  England Premier League
4769	 4769	  France Ligue 1
7809	 7809	  Germany 1. Bundesliga
10257	10257	 Italy Serie A
13274	13274	 Netherlands Eredivisie
15722	15722	 Poland Ekstraklasa
17642	17642	 Portugal Liga ZON Sagres
..........................`



### Subqueries in SELECT
**QUERY** You will construct a query that calculates the average number of goals per match in each country's league.

```SQL
SELECT 
	l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
    (SELECT ROUND(AVG(home_goal + away_goal), 2) 
     FROM match
     WHERE season = '2013/2014') AS overall_avg
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE season = '2013/2014'
GROUP BY l.name;

```
`OUTPUT
league	                    avg_goals	overall_avg
Switzerland Super League	   2.89	    2.77
Poland Ekstraklasa	         2.64	    2.77
Netherlands Eredivisie	     3.20	    2.77`

**NOTE: When writing queries in SELECT, it's important to remember that filtering the main query does not filter the subquery -- and vice versa.**
**QUERY** In the previous exercise, you created a column to compare each league's average total goals to the overall average goals in the 2013/2014 season. In this exercise, you will add a column that directly compares these values by subtracting the overall average from the subquery.
```SQL
SELECT
	-- Select the league name and average goals scored
	name AS league,
	ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Subtract the overall average from the league average
	ROUND(AVG(m.home_goal + m.away_goal) -
		(SELECT AVG(home_goal + away_goal)
		 FROM match 
         WHERE season = '2013/2014'),2) AS diff
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Only include 2013/2014 results
WHERE season = '2013/2014'
GROUP BY l.name;
```


### Subqueries everywhere!
**QUERY** You will build a query that contains three subqueries -- one in the SELECT clause, one in the FROM clause, and one in the WHERE clause. The final query will extract data assessing how the average home and away goals scored changes across match stages. Do teams score more or less as the stakes get higher?

```sQL
SELECT 
	-- Select the stage and average goals for each stage
m.stage,
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Select the average overall goals for the 2012/2013 season
    ROUND((SELECT AVG(home_goal + away_goal) 
           FROM match 
           WHERE season = '2012/2013'),2) AS overall
FROM match AS m
WHERE season = '2012/2013'
GROUP BY m.stage;
```

**QUERY** you will turn the main query into a subquery to extract a list of stages where the average home goals in a stage, is higher than the overall average for home goals in a match.

```SQL
SELECT 
	-- Select the stage and average goals from the subquery
	s.stage,
	ROUND(s.avg_goals,2) AS avg_goals
FROM 
	-- Select the stage and average goals in 2013/2014
	(SELECT
		 stage,
         AVG(home_goal + away_goal) AS avg_goals
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage) AS s
WHERE 
	-- Filter the main query using the subquery
	s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                    FROM match WHERE season = '2012/2013');```

**QUERY** In the previous exercise, you added a subquery to the FROM statement and selected the stages where the number of average goals in a stage exceeded the overall average number of goals in the 2012/2013 match season. In this final step, you will add a subquery in SELECT to compare the average number of goals scored in each stage to the total.
```SQL
SELECT 
	-- Select the stage and average goals from s
	s.stage,
    ROUND(s.avg_goals,2) AS avg_goal,
    -- Select the overall average for 2012/2013
    (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
	-- Select the stage and average goals in 2012/2013 from match
	(SELECT
		 stage,
         AVG(home_goal + away_goal) AS avg_goals
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage) AS s
WHERE 
	-- Filter the main query using the subquery
	s.avg_goals > (SELECT AVg(home_goal+ away_goal) 
                    FROM match WHERE season = '2012/2013');```

### Basic Correlated Subqueries
Correlated subqueries are subqueries that reference one or more columns in the main query. Correlated subqueries depend on information in the main query to run, and thus, cannot be executed on their own.

**QUERY** In this exercise, you will practice using correlated subqueries to examine matches with scores that are extreme outliers for each country -- above 3 times the average score!

```SQL
SELECT 
	-- Select country ID, date, home, and away goals from match
	main.country_id,
    main.date,
    main.home_goal, 
    main.away_goal
FROM match AS main
WHERE 
	-- Filter the main query by the subquery
	(home_goal + away_goal) > 
        (SELECT AVG((sub.home_goal + sub.away_goal) * 3)
         FROM match AS sub
         -- Join the main query to the subquery in WHERE
         WHERE main.country_id = sub.country_id);
```

**QUERY**  In this exercise, you're going to add an additional column for matching to answer the question -- what was the highest scoring match for each country, in each season?
```SQL
SELECT 
	-- Select country ID, season, home, and away goals from match
	main.country_id,
    main.date,
    main.home_goal,
    main.away_goal
FROM match AS main
WHERE 
	-- Filter for matches with the highest number of goals scored
	(home_goal + away_goal) =
        (SELECT MAX(sub.home_goal + sub.away_goal)
         FROM match AS sub
         WHERE main.country_id = sub.country_id
               AND main.season = sub.season);

```

### Nested Subqueries

**QUERY** In this exercise, you will practice creating a nested subquery to examine the highest total number of goals in each season, overall, and during July across all seasons.
```SQL
SELECT
	-- Select the season and max goals scored in a match
	season,
    MAX(home_goal + away_goal) AS max_goals,
    -- Select the overall max goals scored in a match
   (SELECT MAX(home_goal + away_goal) FROM match) AS overall_max,
   -- Select the max number of goals scored in any match in July
   (SELECT MAX(home_goal + away_goal) 
    FROM match
    WHERE id IN (
          SELECT id FROM match WHERE EXTRACT(MONTH FROM date) = 07)) AS july_max
FROM match
GROUP BY season;
```

**QUERY**In this exercise, you will practice creating some more complicated mathematical computations. Your task is to answer a two-part question with a nested subquery in the FROMstatement: What is the average monthly maximum of goals in each season, and which season has the highest monthly max average?
```SQL
SELECT
	-- Select the season and max home, away, total averages
    season,
    MAX(sub2.avg_max_home) AS max_home_avg,
    MAX(sub2.avg_max_away) AS max_away_avg,
    MAX(sub2.total_goals) AS max_total_avg
FROM
	-- Select the season, average max and average total goals
	(SELECT 
     	 sub1.season,
     	 AVG(sub1.max_home) AS avg_max_home, 
     	 AVG(sub1.max_away) AS avg_max_away,
         AVG(sub1.max_home + sub1.max_away) AS total_goals
    FROM
     	-- Select the season, month, and max of home/away goals
     	(SELECT
        	season,
        	EXTRACT(MONTH FROM m.date) AS month,
        	MAX(m.home_goal) AS max_home,
        	MAX(m.away_goal) AS max_away
         FROM match AS m
         GROUP BY season, EXTRACT(MONTH FROM m.date)) AS sub1
    GROUP BY season) AS sub2
GROUP BY season;
```

you generated a list of countries and the number of matches in each country with more than 10 goals overall. The query in that exercise was listed in the FROM statement. In this exercise, you will set up a similar exercise using a CTE.

```SQL
-- Set up your CTE
WITH list AS (
    SELECT 
  		country_id, 
  		id
    FROM match
    WHERE (home_goal + away_goal) >= 10)
-- Select league and count of matches from the CTE
SELECT
    l.name AS league,
    COUNT(list.id) AS matches
FROM league AS l
-- Join the CTE to the league table
LEFT JOIN list ON l.id = list.country_id
GROUP BY l.name;
```
**QUERY** In the previous exercise, you modified a task completed in chapter 2. In this exercise, you will expand on the following exercise looking at details about these matches with very high scores. Just like a subquery in FROM, you can join tables inside a CTE.
```SQL
-- Set up your CTE
with list AS (
  -- Select the league name, date, home, and away goals
    SELECT 
  		l.name AS league, 
     	m.date, 
  		m.home_goal, 
  		m.away_goal,
       (m.home_goal + m.away_goal) AS total_goals
    FROM match AS m
    LEFT JOIN league as l ON m.country_id = l.id)
-- Select the league, name, date, home and away goals from the CTE
SELECT  league, date, home_goal, away_goal
FROM list
-- Filter by total goals
WHERE total_goals >=10;```

**QUERY**  CTEs can help you keep track of the logical flow of your work.
```SQL
-- Set up your CTE
with match_list AS (
    SELECT 
  		country_id,
  	   (home_goal + away_goal) AS goals
    FROM match
  	-- Create a list of match IDs to filter data in the CTE
    WHERE id IN (
       SELECT id
       FROM match 
       WHERE season = '2013/2014' AND EXTRACT(MONTH FROM date) = 08))
-- Select the league name and average of goals in the CTE
SELECT 
	l.name,
    AVG(match_list.goals)
FROM league AS l
-- Join the CTE onto the league table
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY l.name;
```

**QUERY** In the following exercises, you will solve one of the key problems with extracting meaningful information from the European Soccer Database
```SQL
-- Set up your CTE
with home AS (
 	-- Select the match id and team long name
    SELECT
  		m.id, 
  		t.team_long_name 
    FROM match AS m
    LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id)
-- Select the id, team name, and home goals
SELECT 
	home.id,
    home.team_long_name AS hometeam,
    m.home_goal
FROM match AS m
-- Left join the CTE onto the match table
LEFT JOIN home ON m.id = home.id
WHERE m.season = '2014/2015';
```
**QUERY** In this second exercise, you will set up the second CTE joining the away team ID to the team table. This will allow you to create a data set with the away team names included in match records.
```SQL
-- Set up your CTE
with away AS (
 	-- Select the match id and team long name
    SELECT
  		m.id, 
  		t.team_long_name
    FROM match AS m
    LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select the id, team name, and home goals
SELECT 
	m.id,
    away.team_long_name AS awayteam,
    m.away_goal
FROM match AS m
-- Left join the CTE onto the match table
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015';
```
**QUERY** You've now created both CTEs matching the team ID to the home and away team names. Your next step is to combine them and then SELECT them into a main query, giving you a more informative data set.
```SQL
-- Create your home team CTE
with home AS (
    -- Select the match id and team long name
    SELECT m.id, t.team_long_name
    FROM match AS m
    LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id),
-- Create your away team CTE. 
away AS (
    -- Select the match id and team long name
    SELECT m.id, t.team_long_name
    FROM match AS m
    LEFT JOIN team AS t ON m.awayteam_id = t.team_api_id)
-- Select the stage, date, home and away teams
SELECT 
	m.stage,
    m.date,
    home.team_long_name AS home_team,
    away.team_long_name AS away_team,
    m.home_goal,
    m.away_goal
FROM match AS m
-- Join the CTEs onto the match table
LEFT JOIN home ON m.id = home.id
LEFT JOIN away ON m.id = away.id
WHERE m.season = '2014/2015';
```
`OUTPUT
stage	date	   home_team	             away_team	          home_goal	away_goal
1	2014-07-25	  Standard de Liège	     Sporting Charleroi	    3	       0
1	2014-07-26	  Waasland-Beveren	      Club Brugge KV	        0	       2
1	2014-07-26	  KSV Cercle Brugge	     KAA Gent	              0	       0
1	2014-07-26	  Lierse SK	             KV Oostende	           2	       0
1	2014-07-26	  KVC Westerlo	          Sporting Lokeren	      1	       0`

#### The match is OVER
The OVER() clause allows you to pass an aggregate function down a data set, similar to subqueries in SELECT. The OVER() clause offers significant benefits over subqueries in select -- namely, your queries will run faster, and the OVER() clause has a wide range of additional functions and clauses you can include with it that we will cover later on in this chapter.

**QUERY** In this exercise, you will revise some queries from previous chapters using the OVER() clause.

```SQL
SELECT 
	-- Select the id, country name, season, home, and away goals
	m.id, 
    c.name AS country, 
    m.season,
	m.home_goal,
	m.away_goal,
    -- Use a window to include the aggregate average in each row
	AVG(m.home_goal + m.away_goal) OVER() AS overall_avg
FROM match AS m
LEFT JOIN country AS c ON m.country_id = c.id;
```

#### What's OVER here?
Window functions allow you to create a RANK of information according to any variable you want to use to sort your data. When setting this up, you will need to specify what column/calculation you want to use to calculate your rank. This is done by including an ORDER BY clause inside the OVER() clause. Below is an example:
```SQL
SELECT 
    id,
    RANK() OVER(ORDER BY home_goal) AS rank
FROM match;```

**QUERY**In this exercise, you will create a data set of ranked matches according to which leagues, on average, score the most goals in a match.
```SQL
SELECT 
	-- Select the league name and average goals scored
	l.name AS league,
    AVG(m.home_goal + m.away_goal) AS avg_goals,
    -- Rank each league according to the average goals
    RANK() OVER(ORDER BY AVG(m.home_goal + m.away_goal)) AS league_rank
FROM league AS l
LEFT JOIN match AS m 
ON l.id = m.country_id
WHERE m.season = '2011/2012'
GROUP BY l.name
-- Order the query by the rank you created
ORDER BY league_rank;

```
`OUTPUT
league	                   avg_goals	        league_rank
Poland Ekstraklasa	       2.1958333333333333	1
France Ligue 1	           2.5157894736842105	2
Italy Serie A	            2.5837988826815642	3
Switzerland Super League     2.6234567901234568	4
Scotland Premier League	  2.6359649122807018	5
Portugal Liga ZON Sagres     2.6416666666666667	6`




